# Part 3: Fibonacci Example
# Chapter 4: Multiple StepTypes

In [7]:
from dsl import Circuit, StepType
from cb import eq
from util import F

class FiboFirstStep(StepType):
    def setup(self):
        self.c = self.internal("c")
        self.constr(eq(self.circuit.a, 1))
        self.constr(eq(self.circuit.b, 1))
        self.constr(eq(self.circuit.a + self.circuit.b, self.c))
        self.transition(eq(self.circuit.b, self.circuit.a.next()))
        self.transition(eq(self.c, self.circuit.b.next()))

    def wg(self, args):
        a_value, b_value = args
        self.assign(self.circuit.a, F(a_value))
        self.assign(self.circuit.b, F(b_value))
        self.assign(self.c, F(a_value + b_value))

class FiboStep(StepType):
    def setup(self):
        self.c = self.internal("c")
        self.constr(eq(self.circuit.a + self.circuit.b, self.c))
        self.transition(eq(self.circuit.b, self.circuit.a.next()))
        self.transition(eq(self.c, self.circuit.b.next()))

    def wg(self, args):
        a_value, b_value = args
        self.assign(self.circuit.a, F(a_value))
        self.assign(self.circuit.b, F(b_value))
        self.assign(self.c, F(a_value + b_value))

class Fibonacci(Circuit):
    def setup(self):
        self.a = self.forward("a")
        self.b = self.forward("b")
        
        self.fibo_first_step = self.step_type(FiboFirstStep(self, "fibo_first_step"))
        self.fibo_step = self.step_type(FiboStep(self, "fibo_step"))
        self.pragma_num_steps(4)
        self.pragma_first_step(self.fibo_first_step)
        
    def trace(self, args):
        self.add(self.fibo_first_step, (1, 1))
        a = 1
        b = 2
        for i in range(1, 4):
            self.add(self.fibo_step, (a, b))
            prev_a = a
            a = b
            b += prev_a

fibo = Fibonacci()

# success
fibo_witness = fibo.gen_witness(None)
fibo.halo2_mock_prover(fibo_witness)

# fails
evil_witness = fibo_witness.evil_witness_test(step_instance_indices=[0, 0, 1, 1, 2, 2, 3, 3, 3], assignment_indices=[0, 1, 0, 2, 1, 2, 0, 1, 2], rhs=[F(0), F(2), F(2), F(4), F(4), F(6), F(4), F(6), F(10)])
fibo.halo2_mock_prover(evil_witness)

there's last step
4
1
23084509758196191005326514439957645834
Ok(
    (),
)
Err(
    [
        ConstraintCaseDebug {
            constraint: Constraint {
                gate: Gate {
                    index: 0,
                    name: "main",
                },
                index: 3,
                name: "fibo_first_step::(a == 1) => (a + -0x15ebf95182c5551cc8260de4aeb85d5d090ef5a9e111ec87dc5ba0056db1194e) => Product(Fixed { query_index: 0, column_index: 0, rotation: Rotation(0) }, Product(Advice { query_index: 6, column_index: 4, rotation: Rotation(0) }, Sum(Advice { query_index: 1, column_index: 0, rotation: Rotation(0) }, Negated(Constant(0x15ebf95182c5551cc8260de4aeb85d5d090ef5a9e111ec87dc5ba0056db1194e)))))",
            },
            location: InRegion {
                region: Region 0 ('circuit'),
                offset: 0,
            },
            cell_values: [
                (
                    DebugVirtualCell {
                        name: "",
               